# Retreiving and Preparing Stock's Income Statement Information
This section of the code will concentrate on extracting stock's income statement information from IEX Cloud API. At the end, this section of the code will provide a complete and formatted income statement dataset.  

## Importing neccessary modules

In [16]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import xlsxwriter

## Setting up base URL

Fetching the fundamentals and finacial data about the stock from ther IEX Cloud endpoints.

In [17]:
from api_secret import IEX_CLOUD_API_TOKEN

symbol = input("Enther the ticker here: ")

# API endpoints to collect stock data
income_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/income?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
financials_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/financials?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
fundamentals_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/fundamentals?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'

income_endpoint_data = requests.get(income_endpoint).json()
financials_endpoint_data = requests.get(financials_endpoint).json()
fundamentals_endpoint_data = requests.get(fundamentals_endpoint).json()

## Extracting the data
This part will concentrate on extracting the data that will form the final income statement for the stock.

### Forming the Dataset columns
The columns will be comprised of the Pandas DatetimeIndex. It will be created based on the quarters (reportDate) of the reported stock data.


First step is to find the quarters of the reported stock data.

In [18]:
reportDate_list = []
quarter_list = []

# Converting string date into the date object
for stock_data in income_endpoint_data['income']:
    converted_date = datetime.strptime(stock_data['reportDate'], '%Y-%m-%d').date()
    reportDate_list.append(converted_date)


# function to determine to which quarter the reported stock data
# is referred to
def quarter_finder(dates_list):
    global quarter_list

    for d in reportDate_list:
        q1 = date(d.year, 3, 31)
        q2 = date(d.year, 6, 30)
        q3 = date(d.year, 9, 30)
        q4 = date(d.year, 12, 31)

        if d < q1:
            q = date(d.year - relativedelta(year=1).year, 12, 31)
        if d > q1 and d < q2:
            q = date(d.year, 3, 30)
        if d > q2 and d < q3:
            q = date(d.year, 6, 30)
        if d > q3 and d < q4:
            q = date(d.year, 9, 30)
        
        quarter_list.append(q)

quarter_finder(reportDate_list)

The above created Datetime-like array will serve as an input to Pandas' DatetimeIndex objet (columns of the future dataset)

In [19]:
columns = pd.DatetimeIndex(data=quarter_list)

The next step is the creation of the actual DataFrame.

In [20]:
index = [
    'Total Revenue',
    'Total Revenue (Growth)',
    'Cost of Revenue',
    'Cost of Revenue (Growth)',
    'Gross Profit',
    'Gross Profit (Growth)',
    'Operating Expense',
    'SG&A',
    'SG&A (Growth)',
    'Research and Development',
    'Operating Income',
    'Pretax Income',
    'Pretax Income (Growth)',
    'Tax Provision',
    'Net Income',
    'Net Income (Growth)',
    'Basic EPS',
    'Basic EPS (Growth)',
    'Diluted EPS',
    'Total Expenses',
    'Total Expenses (Growth)',
    'Interest Income',
    'Inerest Income (Growth)',
    'Interest Expense',
    'Interest Expense (Growth)',
    'EBIT',
    'EBITDA',
    'EBITDA (Growth)'
]

income_statement = pd.DataFrame(columns=columns, index=index, data=None)

### Filling the dataframe with data returned by API endpoints.

The first API endpoint to get the stock data from is the 'financials' endpoint.

In [21]:
financials_dict = {
    'Total Revenue': [],
    'Operating Income': [],
    'Pretax Income': [],
    'Tax Provision': [],
    'Net Income': [],
    'Total Expenses': [],
    'Interest Income': [],
    'EBIT': [],
    'EBITDA': []
}


# Filling the financials_dict dictionary with data collected
# from the API endpoint
for quarter_data in financials_endpoint_data['financials']:
    financials_dict['Total Revenue'].append(quarter_data['totalRevenue'])
    financials_dict['Operating Income'].append(quarter_data['operatingIncome'])
    financials_dict['Pretax Income'].append(quarter_data['pretaxIncome'])
    financials_dict['Tax Provision'].append(quarter_data['incomeTax'])
    financials_dict['Net Income'].append(quarter_data['netIncome'])
    financials_dict['Total Expenses'].append(quarter_data['operatingExpense'])
    financials_dict['Interest Income'].append(quarter_data['interestIncome'])
    financials_dict['EBIT'].append(quarter_data['ebit'])
    financials_dict['EBITDA'].append(quarter_data['EBITDA'])


# Putting the data into the DataFrame
for stock_attribute in financials_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = financials_dict[stock_attribute]


The second is the income API endpoint.

In [22]:
income_dict = {
    'Cost of Revenue': [],
    'Gross Profit': [],
    'Operating Expense': [],
    'SG&A': [],
    'Research and Development': []
}

for quarter_data in income_endpoint_data['income']:
    income_dict['Cost of Revenue'].append(quarter_data['costOfRevenue'])
    income_dict['Gross Profit'].append(quarter_data['grossProfit'])
    income_dict['Operating Expense'].append(quarter_data['operatingExpense'])
    income_dict['SG&A'].append(quarter_data['sellingGeneralAndAdmin'])
    income_dict['Research and Development'].append(quarter_data['researchAndDevelopment'])

for stock_attribute in income_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = income_dict[stock_attribute]

The final endpoint to use the data from in the dataset is the fundamentals endpoint.

In [23]:
fundamentals_dict = {
    'Basic EPS': [],
    'Diluted EPS': [],
    'Interest Expense': []
}

for quarter_data in fundamentals_endpoint_data['fundamentals']:
    fundamentals_dict['Basic EPS'].append(quarter_data['incomeNetPerWabsoSplitAdjusted'])
    fundamentals_dict['Diluted EPS'].append(quarter_data['incomeNetPerWadsoSplitAdjusted'])
    fundamentals_dict['Interest Expense'].append(quarter_data['expensesInterest'])

for stock_attribute in fundamentals_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = fundamentals_dict[stock_attribute]

## Calculating the Growth

Below section deals with the 'Growth' columns to give an investor an advantage of company's success metric.

In [82]:
# Calculating the growth rate based on the stock metrics' 
for df_index, row_name in enumerate(income_statement.index):
    if 'Growth' in row_name:
        for row_index, value in enumerate(income_statement.iloc[df_index - 1]):
            try:
                growth_rate = (value / income_statement.iloc[df_index - 1, row_index + 1]) * 100
                income_statement.iloc[df_index, row_index] = (growth_rate - 100) / 100
            except IndexError:
                break

income_statement

,2022-09-30,2022-06-30,2022-03-30,2021-12-31,2021-09-30,2021-06-30,2021-03-30,2020-12-31,2020-09-30,2020-06-30,2020-03-30,2019-12-31
Total Revenue,1.200384e+09,1.051042e+09,9.902696e+08,1.008282e+09,8.686378e+08,8.393404e+08,7.362657e+08,7.732302e+08,6.823364e+08,6.156868e+08,5.902660e+08,631836664
Total Revenue (Growth),1.420893e-01,6.136979e-02,-1.786398e-02,1.607617e-01,3.490529e-02,1.399966e-01,-4.780525e-02,1.332096e-01,1.082526e-01,4.306652e-02,-6.579330e-02,
Cost of Revenue,2.921155e+08,2.610646e+08,2.566829e+08,2.349591e+08,2.133742e+08,1.955474e+08,1.621311e+08,1.709210e+08,1.409706e+08,1.405196e+08,1.323196e+08,140963949
Cost of Revenue (Growth),1.189395e-01,1.707037e-02,9.245779e-02,1.011599e-01,9.116360e-02,2.061064e-01,-5.142656e-02,2.124588e-01,3.209017e-03,6.197126e-02,-6.132286e-02,
Gross Profit,8.762159e+08,8.118964e+08,7.088941e+08,7.697957e+08,6.788207e+08,6.338483e+08,5.594246e+08,6.039696e+08,5.124555e+08,4.958939e+08,4.530098e+08,480269456
Gross Profit (Growth),7.922122e-02,1.453000e-01,-7.911395e-02,1.340192e-01,7.095138e-02,1.330361e-01,-7.375368e-02,1.785797e-01,3.339729e-02,9.466503e-02,-5.675917e-02,
Operating Expense,9.191036e+08,8.376789e+08,8.208497e+08,7.547722e+08,7.177701e+08,6.776864e+08,5.917275e+08,5.885823e+08,5.460452e+08,5.199430e+08,5.085164e+08,517703188
SG&A,4.838922e+08,4.721346e+08,4.369315e+08,4.172877e+08,4.018585e+08,3.724033e+08,3.453359e+08,3.303287e+08,3.063400e+08,2.930369e+08,3.023411e+08,298137836
SG&A (Growth),2.490305e-02,8.056908e-02,4.707491e-02,3.839464e-02,7.909494e-02,7.837994e-02,4.543088e-02,7.830750e-02,4.539748e-02,-3.077390e-02,1.409834e-02,
Research and Development,1.356005e+08,1.269821e+08,1.310406e+08,1.145664e+08,1.105887e+08,1.083267e+08,9.981679e+07,9.076080e+07,9.086513e+07,8.475504e+07,8.382062e+07,74721417


Dealing with NaN values.

In [1]:
income_statement.fillna(value='', inplace=True)

NameError: name 'income_statement' is not defined